<a href="https://colab.research.google.com/github/Rob12312368/Machine_Learning/blob/main/DecisionTree_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np
from collections import deque
import time
import sys
import warnings
from scipy.stats import chi2
#from sklearn.metrics import mutual_info_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
class Node:
    def __init__(self,df):
        self.criteria = None
        self.df = df
        self.left = None
        self.right = None

def entropy(df, col):
    zerop = df[df[col] == 0].shape[0] / df.shape[0]
    onep = df[df[col] == 1].shape[0] / df.shape[0]
    if zerop == 0 and onep == 0:
        return 0
    elif zerop == 0:
        return onep * np.log2(1/onep)
    elif onep == 0:
        return zerop * np.log2(1/zerop)
    return zerop * np.log2(1/zerop) + onep * np.log2(1/onep)
def mutual_info(df,col,y):

    hx = entropy(df, col)

    hxy = 0
    for i in range(2):
        for j in range(2):
            #print(df['Sex'] == i & df['Survived'] == j)
            p = df[(df[col] == i) & (df[y] == j)].shape[0] / df.shape[0]
            if p == 0:
                continue
            log = np.log2(df[df[y] == j].shape[0]/df.shape[0]/p)
            #print(p, log)
            hxy += p * log
    return hx - hxy
def best_col(df, x, y):
    biggest = 0
    best = ''
    for col in x:
        if col != y:
            tmp = mutual_info(df,col,y)
            if tmp > biggest:
                biggest = tmp
                best = col
    return (biggest, best)

def decision_tree(node, x, threshold):
    if x == ['Survived'] or node.df.shape[0] < threshold:
        return
    biggest, best = best_col(node.df, x, 'Survived')
    if best == '':
        return
    node.criteria = best
    #print(list(node.df.columns))
    x.remove(best)
    node.left = Node(node.df[node.df[best] == 0])
    node.right = Node(node.df[node.df[best] == 1])

    decision_tree(node.left, x, threshold)
    decision_tree(node.right, x, threshold)
def tree_traversal_bfs(quelist):
    for q in quelist:
        while q:
            line = []
            for i in range(len(q)):
                tmp = q.popleft()
                line.append(tmp.criteria)
                print(tmp.df.shape[0], end=' ')
                #print(tmp.left)
                if tmp.left:
                    q.append(tmp.left)
                if tmp.right:
                    q.append(tmp.right)
            print(line)
        print()

def random_forest(df, n):
    #print(df.columns)
    y = df['Survived']
    X = df.drop(columns = ['Survived'])
    nodes = []
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
        node = Node(pd.concat([X_train, y_train], axis=1))
        nodes.append(node)
        decision_tree(node, list(X_train.columns), X_train.shape[0] * 0.05)
    return nodes
def random_forest_dropfeature(df, n):
    #print(df.columns)
    y = df['Survived']
    X = df.drop(columns = ['Survived'])
    tmpcol = X.columns
    nodes = []
    for i in range(n):
        X = df.drop([tmpcol[i], 'Survived'], axis = 1)
        #print(X.columns)
        node = Node(pd.concat([X, y], axis=1))
        nodes.append(node)
        decision_tree(node, list(X.columns), X.shape[0] * 0.05)
    return nodes
def predict(start,x):
    ans = []
    for s in start:
        cur = s
        while cur.criteria != None:
            if x[cur.criteria] == 0:
                cur = cur.left
            else:
                cur = cur.right
        ans.append(cur.df.mode()['Survived'].values[0])
    #print(ans)
    return pd.Series(ans).mode().values[0]

def cross_valid(df,k):
    bound = np.linspace(0, df.shape[0], k+1, dtype=int)
    average = 0
    for i in range(len(bound)-1):
        test = df.iloc[bound[i]:bound[i+1]]
        train = df.drop(index = [i for i in range(bound[i], bound[i+1])])
        #print(test.shape[0], train.shape[0])
        node = Node(train)
        decision_tree(node, list(train.columns), train.shape[0] * 0.05)
        predictions = []
        for i in range(test.shape[0]):
            predictions.append(predict([node],test.iloc[i]))
        average += accuracy_score(test['Survived'], predictions)
    print(average / k)

def cross_valid_forest(df,k):
    bound = np.linspace(0, df.shape[0], k+1, dtype=int)
    average = 0
    for i in range(len(bound)-1):
        test = df.iloc[bound[i]:bound[i+1]]
        train = df.drop(index = [i for i in range(bound[i], bound[i+1])])
        #print(test.shape[0], train.shape[0])
        nodes = random_forest(train, 5)
        predictions = []
        for i in range(test.shape[0]):
            predictions.append(predict(nodes,test.iloc[i]))
        #print(predictions)
        average += accuracy_score(test['Survived'], predictions)
    print(average / k)

def cross_valid_forest_dropfeature(df,k):
    bound = np.linspace(0, df.shape[0], k+1, dtype=int)
    average = 0
    for i in range(len(bound)-1):
        test = df.iloc[bound[i]:bound[i+1]]
        train = df.drop(index = [i for i in range(bound[i], bound[i+1])])
        #print(test.shape[0], train.shape[0])
        nodes = random_forest_dropfeature(train, 5)
        predictions = []
        for i in range(test.shape[0]):
            predictions.append(predict(nodes,test.iloc[i]))
        #print(predictions)
        average += accuracy_score(test['Survived'], predictions)
    print(average / k)
# # read csv file
# df = pd.read_csv('titanic_data.csv', header=0)

# continuous = ['Age','Fare']
# categorical = df.drop(columns=['Age','Fare']).columns

# for col in continuous:
#     df[col] = (df[col] > df[col].mean()).astype('int')

# df['Pclass'] = (df['Pclass'] > 1).astype('int')
# df['Siblings/Spouses Aboard'] = (df['Siblings/Spouses Aboard'] > df['Siblings/Spouses Aboard'].median()).astype('int')
# df['Parents/Children Aboard'] = (df['Parents/Children Aboard'] > df['Parents/Children Aboard'].median()).astype('int')

# #print(best_col(df, df.columns, 'Survived'))
# print(mutual_info(df, 'Sex', 'Survived'))
# node = Node(df)
# decision_tree(node, list(df.columns), df.shape[0] * 0.05)
# tree_traversal_bfs([deque([node])])

# #print(df.iloc[0]['Sex'])
# #print(predict(node,df.iloc[0]))

# cross_valid(df,10)
# print(df.columns)
# #print(dict(zip(list(df.columns)[1:],[1,0,0,1,0,1])))
# x = pd.Series(dict(zip(list(df.columns)[1:],[1,0,0,1,0,1])))
# print(x)
# print(predict([node],x))


# #forest = random_forest(df, 5)
# #quelist = [deque([nodes]) for nodes in forest]
# #tree_traversal_bfs(quelist)

# #cross_valid_forest(df,10)

# forest = random_forest_dropfeature(df, 6)
# quelist = [deque([nodes]) for nodes in forest]
# tree_traversal_bfs(quelist)

# cross_valid_forest_dropfeature(df, 10)


# 4.1 - Data Preprocessing

In [33]:
df = pd.read_csv('titanic_data.csv', header=0)

continuous = ['Age','Fare']
categorical = df.drop(columns=['Age','Fare']).columns

for col in continuous:
    df[col] = (df[col] > df[col].mean()).astype('int')

df['Pclass'] = (df['Pclass'] > 1).astype('int')
df['Siblings/Spouses Aboard'] = (df['Siblings/Spouses Aboard'] > df['Siblings/Spouses Aboard'].median()).astype('int')
df['Parents/Children Aboard'] = (df['Parents/Children Aboard'] > df['Parents/Children Aboard'].median()).astype('int')

# 4.2 - Mutual Information

In [34]:
def mutual_info(df,col,y):

    hx = entropy(df, col)

    hxy = 0
    for i in range(2):
        for j in range(2):
            #print(df['Sex'] == i & df['Survived'] == j)
            p = df[(df[col] == i) & (df[y] == j)].shape[0] / df.shape[0]
            if p == 0:
                continue
            log = np.log2(df[df[y] == j].shape[0]/df.shape[0]/p)
            #print(p, log)
            hxy += p * log
    return hx - hxy
print(mutual_info(df, 'Sex', 'Survived'))

0.2168495048312652


# 4.3 - Building Decison Tree (from scratch)

In [35]:
def decision_tree(node, x, threshold):
    if x == ['Survived'] or node.df.shape[0] < threshold:
        return
    biggest, best = best_col(node.df, x, 'Survived')
    if best == '':
        return
    node.criteria = best
    #print(list(node.df.columns))
    x.remove(best)
    node.left = Node(node.df[node.df[best] == 0])
    node.right = Node(node.df[node.df[best] == 1])

    decision_tree(node.left, x, threshold)
    decision_tree(node.right, x, threshold)
node = Node(df)
decision_tree(node, list(df.columns), df.shape[0] * 0.05)


# 4.4 - Tree Traversal

In [36]:
def tree_traversal_bfs(quelist):
  for q in quelist:
      while q:
          line = []
          for i in range(len(q)):
              tmp = q.popleft()
              line.append(tmp.criteria)
              print(tmp.df.shape[0], end=' ')
              #print(tmp.left)
              if tmp.left:
                  q.append(tmp.left)
              if tmp.right:
                  q.append(tmp.right)
          print(line)
tree_traversal_bfs([deque([node])])

887 ['Sex']
573 314 ['Pclass', None]
122 451 ['Age', 'Parents/Children Aboard']
27 95 381 70 [None, 'Siblings/Spouses Aboard', None, None]
71 24 ['Fare', None]
43 28 [None, None]


# 4.5 - 10 fold Cross Validation

In [37]:
def cross_valid(df,k):
    bound = np.linspace(0, df.shape[0], k+1, dtype=int)
    average = 0
    for i in range(len(bound)-1):
        test = df.iloc[bound[i]:bound[i+1]]
        train = df.drop(index = [i for i in range(bound[i], bound[i+1])])
        #print(test.shape[0], train.shape[0])
        node = Node(train)
        decision_tree(node, list(train.columns), train.shape[0] * 0.05)
        predictions = []
        for i in range(test.shape[0]):
            predictions.append(predict([node],test.iloc[i]))
        average += accuracy_score(test['Survived'], predictions)
    print(average / k)
cross_valid(df,10)

0.7790474974463739


# 4.6 - Will I Survive?

In [38]:
def predict(start,x):
    ans = []
    for s in start:
        cur = s
        while cur.criteria != None:
            if x[cur.criteria] == 0:
                cur = cur.left
            else:
                cur = cur.right
        ans.append(cur.df.mode()['Survived'].values[0])
    #print(ans)
    return pd.Series(ans).mode().values[0]
x = pd.Series(dict(zip(list(df.columns)[1:],[1,0,0,1,0,1])))
print(x)
print(predict([node],x))

Pclass                     1
Sex                        0
Age                        0
Siblings/Spouses Aboard    1
Parents/Children Aboard    0
Fare                       1
dtype: int64
0


# 4.7 - Random Forest (5 trees)

## a - Forest Display

In [39]:
def random_forest(df, n):
    #print(df.columns)
    y = df['Survived']
    X = df.drop(columns = ['Survived'])
    nodes = []
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
        node = Node(pd.concat([X_train, y_train], axis=1))
        nodes.append(node)
        decision_tree(node, list(X_train.columns), X_train.shape[0] * 0.05)
    return nodes
forest = random_forest(df, 5)
quelist = [deque([nodes]) for nodes in forest]
tree_traversal_bfs(quelist)

709 ['Sex']
456 253 ['Pclass', None]
97 359 ['Age', 'Parents/Children Aboard']
23 74 310 49 [None, 'Fare', None, None]
32 42 [None, 'Siblings/Spouses Aboard']
25 17 [None, None]
709 ['Sex']
465 244 ['Pclass', None]
101 364 ['Age', None]
19 82 [None, 'Siblings/Spouses Aboard']
61 21 ['Fare', None]
38 23 ['Parents/Children Aboard', None]
37 1 [None, None]
709 ['Sex']
463 246 ['Pclass', None]
99 364 ['Age', None]
17 82 [None, 'Siblings/Spouses Aboard']
58 24 ['Fare', None]
38 20 ['Parents/Children Aboard', None]
37 1 [None, None]
709 ['Sex']
458 251 ['Pclass', None]
97 361 ['Age', 'Parents/Children Aboard']
23 74 307 54 [None, 'Fare', None, None]
33 41 [None, 'Siblings/Spouses Aboard']
22 19 [None, None]
709 ['Sex']
460 249 ['Pclass', None]
100 360 ['Siblings/Spouses Aboard', None]
72 28 ['Fare', None]
39 33 ['Parents/Children Aboard', None]
38 1 ['Age', None]
4 34 [None, None]


## b - Cross Validation On the Forest

In [40]:
def cross_valid_forest(df,k):
    bound = np.linspace(0, df.shape[0], k+1, dtype=int)
    average = 0
    for i in range(len(bound)-1):
        test = df.iloc[bound[i]:bound[i+1]]
        train = df.drop(index = [i for i in range(bound[i], bound[i+1])])
        #print(test.shape[0], train.shape[0])
        nodes = random_forest(train, 5)
        predictions = []
        for i in range(test.shape[0]):
            predictions.append(predict(nodes,test.iloc[i]))
        #print(predictions)
        average += accuracy_score(test['Survived'], predictions)
    print(average / k)
cross_valid_forest(df,10)

0.7801838610827374


## c - Will I Survive?

In [41]:
print(predict(forest,x))

0


# 4.8 Random Forest (6 trees)

## a - Forest Display

In [42]:
forest = random_forest_dropfeature(df, 6)
quelist = [deque([nodes]) for nodes in forest]
tree_traversal_bfs(quelist)

887 ['Sex']
573 314 ['Fare', None]
466 107 ['Parents/Children Aboard', None]
417 49 ['Siblings/Spouses Aboard', None]
371 46 ['Age', None]
199 172 [None, None]
887 ['Pclass']
216 671 ['Fare', None]
57 159 ['Age', 'Siblings/Spouses Aboard']
8 49 80 79 [None, 'Parents/Children Aboard', None, None]
48 1 [None, None]
887 ['Sex']
573 314 ['Pclass', None]
122 451 ['Siblings/Spouses Aboard', None]
88 34 ['Fare', None]
49 39 ['Parents/Children Aboard', None]
48 1 [None, None]
887 ['Sex']
573 314 ['Pclass', None]
122 451 ['Age', None]
27 95 [None, 'Fare']
43 52 [None, 'Parents/Children Aboard']
39 13 [None, None]
887 ['Sex']
573 314 ['Pclass', None]
122 451 ['Age', None]
27 95 [None, 'Siblings/Spouses Aboard']
71 24 ['Fare', None]
43 28 [None, None]
887 ['Sex']
573 314 ['Pclass', None]
122 451 ['Age', None]
27 95 [None, 'Siblings/Spouses Aboard']
71 24 ['Parents/Children Aboard', None]
64 7 [None, None]


## b - Cross Validation on the Forest

In [43]:
cross_valid_forest_dropfeature(df, 10)

0.7790474974463739


## c - Will I Survive?

In [44]:
print(predict(forest,x))

0
